# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
cities_df = pd.read_csv('../output_data/cities.csv',index_col=[0])

cities_df

,City,Temperature,Latitude,Longitude,Humidity,Cloudiness,Wind Speed (mph)
0,puerto ayora,25.56,-0.7393,-90.3518,91,53,2.50
1,gannan,-5.45,47.9147,123.5031,90,15,2.80
2,saskylakh,-31.79,71.9167,114.0833,82,0,3.98
3,ushuaia,17.00,-54.8000,-68.3000,72,0,2.06
4,cidreira,23.68,-30.1811,-50.2056,80,94,7.99
...,...,...,...,...,...,...,...
562,bardiyah,15.64,31.7561,25.0865,43,0,4.07
563,chimoio,20.76,-19.1164,33.4833,94,40,2.57
564,eureka,8.50,40.8021,-124.1637,91,95,0.99
565,marrakesh,13.00,31.6315,-8.0083,94,75,2.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Latitude", "Longitude"]].astype(float)

humidity = cities_df["Humidity"].astype(float)

max_humidity = cities_df["Humidity"].max()

In [20]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius = 3)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# narrow down to cities that meet desired criteria - note that the below temperatures are in Celcius 
# converted to accommodate data already pulled from API

narrowed_cities_df = cities_df.loc[(cities_df['Temperature'] > 21.1111) & 
                                   (cities_df['Temperature'] < 26.6667) & 
                                   (cities_df['Wind Speed (mph)'] < 10) &
                                   (cities_df['Cloudiness'] == 0)]

# drop NA values
narrowed_cities_df.dropna()



,City,Temperature,Latitude,Longitude,Humidity,Cloudiness,Wind Speed (mph)
20,pisco,21.62,-13.7000,-76.2167,78,0,3.60
34,arraial do cabo,22.98,-22.9661,-42.0278,86,0,2.50
87,new norfolk,21.33,-42.7826,147.0587,40,0,0.45
136,saint-philippe,24.50,-21.3585,55.7679,61,0,1.54
189,salvador,23.00,13.8333,-88.9167,83,0,1.54
203,saint-pierre,24.66,-21.3393,55.4781,61,0,1.54
280,gao,22.21,16.6362,1.6370,17,0,4.61
282,neuquen,25.00,-38.9516,-68.0591,50,0,2.57
293,saint-joseph,24.40,-21.3667,55.6167,61,0,1.54
310,springbok,21.12,-29.6643,17.8865,32,0,1.79


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = narrowed_cities_df.dropna()

In [23]:
# combine lat and longs for search in the Google Places API
lats = hotel_df['Latitude']
lngs = hotel_df['Longitude']

search_locations= hotel_df["Latitude"].astype(str) + "," + hotel_df["Longitude"].astype(str)

search_locations

20                 -13.7,-76.2167
34              -22.9661,-42.0278
87              -42.7826,147.0587
136              -21.3585,55.7679
189              13.8333,-88.9167
203              -21.3393,55.4781
280    16.6362,1.6369999999999998
282             -38.9516,-68.0591
293              -21.3667,55.6167
310              -29.6643,17.8865
320               20.1833,94.8833
388              26.1004,-80.3998
420               17.0151,54.0924
443               21.4267,39.8261
446               -12.1167,-77.05
514               27.9167,96.1667
dtype: object

In [25]:
hotel_name = []
hotel_lat = []
hotel_lon = []
cities_without_hotels = []
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for loc in search_locations:
    
    # geocoordinates
    target_search = "Hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": loc,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    response = requests.get(base_url, params=params)
    hotels_data = response.json()
    
    if (hotels_data['results'] == []):
         cities_without_hotels.append(loc)
    else:
        hotel_name.append(hotels_data['results'][0]['name'])
        hotel_lat.append(hotels_data['results'][0]['geometry']['location']['lat'])
        hotel_lon.append(hotels_data['results'][0]['geometry']['location']['lng'])

In [26]:
hotels_data

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 27.9282248, 'lng': 96.16381559999999},
    'viewport': {'northeast': {'lat': 27.92944867989272,
      'lng': 96.16510377989272},
     'southwest': {'lat': 27.92674902010728, 'lng': 96.16240412010727}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Hotel Shivam',
   'photos': [{'height': 2304,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117011351612993721224">A Google User</a>'],
     'photo_reference': 'ATtYBwJS7dj_TiWIgVDg-9x_DaLPbbmLH8utaSN7lTX_HMBMdYbalOK-DKCCBUlRGDaqBiSL_AV5phWOq8AcBfbZdToxy89DJFmHBeVcpODEwF9kIz_BpTO1c7kHxMwmpgwFUbWTglDyzmXhsOm2BjLr4M0SZZn0BVZofNlfGoLC_MaVJLnW',
     'width': 4608}],
   'place_id': 'ChIJJRkLVM1lPjcRdKOC6viOa5g',
   'plus_code': {'compound_code': 'W5H7+7G Tezu, Arunachal Pradesh, India',
    'global_code': '7MVRW5H7+7G'},
   'rating': 3.5,
   'reference': '

In [27]:
#create new dataframe with hotel results
hotel_dict = {
    "Hotel Name": hotel_name,
    "Hotel Latitude": hotel_lat,
    "Hotel Longitude": hotel_lon
    }
#write to a dataframe
nearby_hotels_df = pd.DataFrame(hotel_dict)

nearby_hotels_df

,Hotel Name,Hotel Latitude,Hotel Longitude
0,Apart Hotel Sol de Paracas,-13.717352,-76.209877
1,Orlanova Boutique Hotel,-22.971534,-42.021067
2,Tynwald Willow Bend Estate,-42.775558,147.076676
3,Les Embruns Du Baril,-21.369201,55.727949
4,LODGE PALMAE,-21.296301,55.499885
5,Hilton Garden Inn Neuquen,-38.941906,-68.059576
6,Village Sauvage,-21.379092,55.659249
7,Annie's Cottage - best b&b in Springbok,-29.662195,17.882485
8,Man Thi Tar Hotel Minbu,20.165208,94.878630
9,Bonaventure Resort & Spa,26.124906,-80.377159


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
"""
#<dt>City</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
#</dl>

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in nearby_hotels_df.iterrows()]
locations = nearby_hotels_df[["Hotel Latitude", "Hotel Longitude"]]

In [33]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info
)

# Display figure

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))